In [ ]:
import sqlite3
import os
import time
import tkinter as tk
from tkinter import ttk, messagebox
import threading
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from serial import Serial, SerialException
import numpy as np
import math
import joblib
import pywt
from scipy.signal import find_peaks
from sklearn.preprocessing import StandardScaler
from scipy.stats import iqr
import pandas as pd  # Importar pandas

# Cargar el modelo entrenado
modelo_entrenado = joblib.load(r"C:\Users\ssote\OneDrive\Desktop\LECG\MACHINE LEARNING\modelo_entrenado (5).pkl")

# Función para inicializar la base de datos
def inicializar_base_datos():
    if not os.path.exists('datos.db'):
        conn = sqlite3.connect('datos.db')
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS variables (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                ecg_data_D1 REAL,
                ecg_data_D2 REAL,
                ecg_data_D3 REAL
            )
        ''')
        conn.commit()
        conn.close()
        print("Base de datos inicializada.")
    else:
        print("Base de datos ya existe.")

# Función para insertar datos
def insertar_datos(ecg_data_D1, ecg_data_D2, ecg_data_D3):
    conn = sqlite3.connect('datos.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO variables (ecg_data_D1, ecg_data_D2, ecg_data_D3)
        VALUES (?, ?, ?)
    ''', (ecg_data_D1, ecg_data_D2, ecg_data_D3))
    conn.commit()
    conn.close()

# Función para consultar datos
def consultar_datos():
    conn = sqlite3.connect('datos.db')
    cursor = conn.cursor()
    cursor.execute('SELECT ecg_data_D1, ecg_data_D2, ecg_data_D3 FROM variables')
    datos = cursor.fetchall()
    conn.close()
    return datos

# Función para consultar datos
def consultar_datos_derivacion(derivacion):
    conn = sqlite3.connect('datos.db')
    cursor = conn.cursor()
    cursor.execute(f'SELECT ecg_data_D{derivacion} FROM variables')
    datos = cursor.fetchall()
    conn.close()
    return [dato[0] for dato in datos]

# Función para preprocesar la señal usando DWT
def preprocess_signal_with_dwt(signal, wavelet='db6'):
    filter_length = pywt.Wavelet(wavelet).dec_len
    max_level = pywt.dwt_max_level(len(signal), filter_length)
    coeffs = pywt.wavedec(signal, wavelet, level=max_level)
    if len(coeffs) > 1:
        coeffs[1] = np.zeros_like(coeffs[1])  # D1
    if len(coeffs) > 2:
        coeffs[2] = np.zeros_like(coeffs[2])  # D2
    coeffs[0] = np.zeros_like(coeffs[0])  # A_max_level
    cleaned_signal = pywt.waverec(coeffs, wavelet)
    return cleaned_signal

# Función para detectar picos R y calcular los intervalos R-R
def calculate_rr_intervals(signal, fs=100):
    peaks, _ = find_peaks(signal, distance=fs/2)
    rr_intervals = np.diff(peaks) / fs
    return rr_intervals

# Función para calcular la media móvil
def running_mean(rr_intervals):
    rmean = np.zeros(len(rr_intervals))
    rmean[0] = rr_intervals[0]
    for i in range(1, len(rr_intervals)):
        rmean[i] = 0.75 * rmean[i-1] + 0.25 * rr_intervals[i]
    return rmean

# Función para clasificar los intervalos R-R
def classify_rr_intervals(rr_intervals, rmean):
    classifications = np.zeros(len(rr_intervals))
    for i in range(len(rr_intervals)):
        if rr_intervals[i] < 0.85 * rmean[i]:
            classifications[i] = 0  # Corto
        elif rr_intervals[i] > 1.15 * rmean[i]:
            classifications[i] = 2  # Largo
        else:
            classifications[i] = 1  # Regular
    return classifications

# Función para generar las características de la matriz de transición
def generate_transition_matrix_features(rr_intervals):
    if len(rr_intervals) < 2:
        return np.zeros(9)
    rmean = running_mean(rr_intervals)
    classifications = classify_rr_intervals(rr_intervals, rmean)
    transition_matrix = np.zeros((3, 3))
    for i in range(len(classifications) - 1):
        current_class = int(classifications[i])
        next_class = int(classifications[i + 1])
        transition_matrix[current_class, next_class] += 1
    return transition_matrix.flatten()

# Función para calcular diversas características a partir de los intervalos R-R
def extract_features(signal):
    rr_intervals = calculate_rr_intervals(signal)
    features = []
    features.extend(generate_transition_matrix_features(rr_intervals))
    features.append(np.mean(np.abs(np.diff(rr_intervals)) / np.mean(rr_intervals)))
    features.append(np.sum(np.abs(np.diff(rr_intervals)) > 0.2))
    features.append(np.sqrt(np.mean(np.square(np.diff(rr_intervals)))))
    features.append(np.std(rr_intervals))
    features.append(np.median(np.abs(rr_intervals - np.median(rr_intervals))))
    features.append(np.std(rr_intervals) / np.mean(rr_intervals))
    features.append(iqr(rr_intervals))
    features.append(np.max(rr_intervals) - np.min(rr_intervals))
    features.append((2 / len(rr_intervals)) * np.sum((np.arange(1, len(rr_intervals)+1) - np.cumsum(np.sort(rr_intervals)) / np.sum(rr_intervals))) / len(rr_intervals))
    return features

# Función para seleccionar solo las características necesarias
def select_needed_features(features):
    needed_indices = [1, 2, 5, 8]  # Índices de 'ShorttoRegular', 'RegulartoLong', 'LongtoLong', 'RRvar'
    return [features[i] for i in needed_indices]

# Función principal para procesar una nueva señal y predecir
def predict_new_signal(signal):
    preprocessed_signal = preprocess_signal_with_dwt(signal)
    features = extract_features(preprocessed_signal)
    selected_features = select_needed_features(features)
    feature_names = ['ShorttoRegular', 'RegulartoLong', 'LongtoLong', 'RRvar']
    features_df = pd.DataFrame([selected_features], columns=feature_names)
    prediction = modelo_entrenado.predict(features_df)

    # Interpretar la predicción (1 = AF, 0 = Healthy)
    return prediction[0]

# Inicializar la base de datos
inicializar_base_datos()

# Clase para la interfaz de ECG
class InterfazECG:
    def __init__(self, root):
        self.root = root
        self.root.title("LECG: Aprende practicando")
        self.root.configure(bg='azure')  # Color de fondo rosa pastel
        # Configurar la ventana para abarcar toda la pantalla
        self.width = root.winfo_screenwidth()
        self.height = root.winfo_screenheight() - 100
        self.root.geometry(f'{self.width}x{self.height}+0+0')
        self.datos_recibidos = True
        self.no_data_count = 0
        self.no_data_threshold = 5  # Número de iteraciones sin datos antes de considerar que no se están recibiendo datos

        style = ttk.Style()
        style.configure('TFrame', background='azure')
        style.configure('TLabel', background='azure')
        style.configure('TButton', background='azure', padding=10, font=('Arial', 12))
        
        # Frame de Login
        self.frame_login = ttk.Frame(self.root, style='TFrame')
        self.frame_login.pack(expand=True, fill='both')

        self.label_bienvenida = ttk.Label(self.frame_login, text="¡Welcome to LECG: Learn ECG!", style='TLabel', font=("Arial", 40))
        self.label_bienvenida.pack(pady=20)

        self.label_id = ttk.Label(self.frame_login, text="Enter your ID:", style='TLabel', font=("Arial", 20))
        self.label_id.pack(pady=10)

        self.entry_id = ttk.Entry(self.frame_login)
        self.entry_id.pack(pady=50, padx=100)

        self.button_login = ttk.Button(self.frame_login, text="Start", command=self.validar_id, style='TButton')
        self.button_login.pack(pady=20)

        self.frame_main = None
        self.canvas = None
        self.fig = None
        self.frame_final = None
        self.frame_buttons = None
        self.prediction = None
        self.frame_bpm = None
        self.bpm_label = None

        # Configuración de Bluetooth Serial
        try:
            self.SerialBT = Serial('COM7', 115200, timeout=1)  # Cambiar por el puerto Bluetooth correcto y la velocidad adecuada
        except SerialException as se:
            print("Error de conexión Bluetooth:", se)
            self.SerialBT = None

        # Ejecutar la lectura de datos en un hilo separado si la conexión Bluetooth está activa
        if self.SerialBT:
            threading.Thread(target=self.leer_datos_serial, daemon=True).start()

    # Función para validar el ID del usuario
    def validar_id(self):
        user_id = self.entry_id.get()
        if user_id.isdigit() and int(user_id) > 0:
            messagebox.showinfo("Access granted", f"Welcome, User {user_id}")
            self.frame_login.pack_forget()
            self.iniciar_interfaz_principal()
        else:
            messagebox.showerror("Error", "Invalid ID. Please enter a positive numeric ID.")

    # Función para iniciar la interfaz principal con gráficas en tiempo real
    def iniciar_interfaz_principal(self):
        self.frame_main = ttk.Frame(self.root, style='TFrame')
        self.frame_main.pack(expand=True, fill='both')

        self.root.title("Gráfica de las tres derivaciones en tiempo real")

        # Frame para la gráfica
        self.frame_grafica = ttk.Frame(self.frame_main, style='TFrame')
        self.frame_grafica.pack(expand=True, fill='both')

        # Crear la figura para la gráfica
        self.fig, (self.ax1, self.ax2, self.ax3) = plt.subplots(3, 1, figsize=(6, 8))
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.frame_grafica)
        self.canvas.get_tk_widget().pack(expand=True, fill='both')

        # Iniciar la actualización de la gráfica
        self.root.after(0, self.actualizar_grafica)

    # Función para leer datos del puerto serial y guardarlos en la base de datos
    def leer_datos_serial(self):
        while True:
            print(f"No data count: {self.no_data_count}")
            try:
                line = self.SerialBT.readline().decode('utf-8').strip()
                if line:
                    print(f"Linea leída: {line}")  # Añade este print para verificar que se están leyendo líneas
                    data = line.split(',')
                    if len(data) == 3:  # Asumiendo que recibes 3 valores separados por coma
                        print("Datos recibidos:", data)
                        insertar_datos(data[0], data[1], data[2])
                        self.no_data_count = 0  # Reiniciar el contador de no datos
                    else:
                        self.no_data_count += 1
                else:
                    self.no_data_count += 1
            except SerialException as se:
                print("Error de conexión Bluetooth:", se)
                self.no_data_count += 1
            except Exception as e:
                print("Error leyendo datos Bluetooth:", e)
                self.no_data_count += 1

            if self.no_data_count >= self.no_data_threshold:
                self.datos_recibidos = False
                self.root.after(0, self.mostrar_interfaz_final)
                break

            time.sleep(0.1)  # Aumenta ligeramente el tiempo de espera para evitar el uso excesivo de la CPU

    # Función para actualizar la gráfica en tiempo real
    def actualizar_grafica(self):
        if self.datos_recibidos:
            datos = consultar_datos()
            if datos:
                ecg_data_D1, ecg_data_D2, ecg_data_D3 = zip(*datos)
                self.ax1.clear()
                self.ax2.clear()
                self.ax3.clear()
                self.ax1.plot(ecg_data_D1, label='ECG D1', color='blue')
                self.ax2.plot(ecg_data_D2, label='ECG D2', color='green')
                self.ax3.plot(ecg_data_D3, label='ECG D3', color='red')
                self.ax1.legend()
                self.ax2.legend()
                self.ax3.legend()
                self.canvas.draw()

                # Programar la siguiente actualización en el hilo principal de tkinter
                self.root.after(1000, self.actualizar_grafica)

    # Función para mostrar la interfaz final con BPM y otros botones
    def mostrar_interfaz_final(self):
        # Limpiar la interfaz principal
        if self.frame_main:
            self.frame_main.pack_forget()

        # Destruir la figura y el canvas del gráfico en tiempo real
        if self.canvas:
            self.canvas.get_tk_widget().pack_forget()
            self.canvas.get_tk_widget().destroy()
            self.canvas = None

        if self.fig:
            plt.close(self.fig)
            self.fig = None

        self.root.title("What abnormality does this ECG present?")

        # Crear el frame para la interfaz final
        self.frame_final = ttk.Frame(self.root, style='TFrame')
        self.frame_final.pack(expand=True, fill='both')

        # Crear la figura para el gráfico reconstruido
        self.fig = plt.figure(figsize=(12, 6))
        ax = self.fig.add_subplot(111)

        # Frame para mostrar el BPM
        self.frame_bpm = ttk.Frame(self.frame_final, style='TFrame')
        self.frame_bpm.pack(pady=10)

        self.bpm_label = ttk.Label(self.frame_bpm, text="BPM: --", style='TLabel', font=("Arial", 25))
        self.bpm_label.pack()

        # Calcular y mostrar el BPM
        self.root.after(0, self.calcular_y_mostrar_bpm)

        # Obtener la señal D2 para la predicción
        datos = consultar_datos_derivacion(2)
        self.prediction = predict_new_signal(datos)

        # Frame para los botones
        self.frame_buttons = ttk.Frame(self.frame_final, style='TFrame')
        self.frame_buttons.pack(pady=20)

        boton_taqui = ttk.Button(self.frame_buttons, text="Atrial Fibrilation", command=self.verificar_respuesta, style='TButton')
        boton_taqui.pack(side='left', padx=50, pady=15)

        boton_bradi = ttk.Button(self.frame_buttons, text="Other anomaly", command=self.verificar_respuesta, style='TButton')
        boton_bradi.pack(side='left', padx=50, pady=15)

        boton_sano = ttk.Button(self.frame_buttons, text="Healthy patient", command=self.verificar_respuesta, style='TButton')
        boton_sano.pack(side='left', padx=50, pady=15)

        boton_no_se = ttk.Button(self.frame_buttons, text="Cannot be determined", command=self.verificar_respuesta, style='TButton')
        boton_no_se.pack(side='left', padx=50, pady=15)

        if datos:
            ax.plot(datos, label='D2', color='deep pink')
            ax.set_facecolor("azure")
            ax.legend()
            ax.set_title("ECG")

        # Crear el canvas y empaquetarlo en el frame_final
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.frame_final)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(expand=True, fill='both')

        # Función para verificar la respuesta seleccionada por el usuario
    def verificar_respuesta(self):
        sender = self.root.focus_get()
        if sender['text'] == "Atrial Fibrilation" and self.prediction == 1:
            messagebox.showinfo("Atrial Fibrilation", "Correcto +10 puntos")
        elif sender['text'] == "Healthy patient" and self.prediction == 0:
            messagebox.showinfo("Healthy patient", "Correcto +10 puntos")
        else:
            messagebox.showinfo("Incorrecto", "Incorrecto, intente de nuevo")

    # Función para calcular y mostrar el BPM
    def calcular_y_mostrar_bpm(self, fs=100):
        datos_D2 = consultar_datos_derivacion(2)
        if datos_D2:
            ecg_data_D2 = [d for d in datos_D2]
            peaks, _ = find_peaks(ecg_data_D2, height=2.0)  # Ajustar la distancia mínima entre picos si es necesario
            rr_intervals = np.diff(peaks)  # Calcular los intervalos RR
            rr_intervals = rr_intervals / fs
            bpm = math.trunc(60.0 / np.mean(rr_intervals))
            self.bpm_label.config(text=f"BPM obtenido: {bpm:.2f}")

# Crear la aplicación Tkinter
root = tk.Tk()
app = InterfazECG(root)
root.mainloop()
